In [13]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import os
import random
from sklearn.utils import check_random_state
from sklearn.model_selection import LeaveOneOut, KFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.feature_selection import SelectKBest, f_classif, RFE, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import LeaveOneOut

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning, message="A single label was found in 'y_true' and 'y_pred'.")
warnings.filterwarnings("ignore", category=UserWarning, message="y_pred contains classes not in y_true")
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# Random seed
RANDOM_SEED = 1492

# Implementation based on: https://machinelearningmastery.com/how-to-configure-k-fold-cross-validation/

#### Part 1: Cleveland Heart Attack Data

In [14]:
def load_and_preprocess_data(file_path, target_column, test_size=0.2, random_state=RANDOM_SEED):
    data = pd.read_csv(file_path)
    X = data.drop(target_column, axis=1)
    y = data[target_column]
    
    # Convert target to binary classification if not already
    if y.nunique() > 2:
        y = y.apply(lambda x: 1 if x > y.median() else 0)
    
    X = X.replace('?', np.nan).astype(float)
    X = X.fillna(X.mean())
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X = pd.DataFrame(X_scaled, columns=X.columns)
    
    if test_size > 0:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
        return X_train, X_test, y_train, y_test
    else:
        return X, y

In [15]:
def set_seed(seed=RANDOM_SEED):
    random.seed(seed)
    np.random.seed(seed)
    check_random_state(seed)

In [16]:
def select_features(X, y, k, method='univariate'):
    if method == 'univariate':
        selector = SelectKBest(f_classif, k=k)
    elif method == 'mutual_info':
        selector = SelectKBest(mutual_info_classif, k=k)
    elif method == 'rfe':
        selector = RFE(estimator=LogisticRegression(max_iter=1000, random_state=RANDOM_SEED), n_features_to_select=k)
    elif method == 'random_forest':
        rf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_SEED)
        rf.fit(X, y)
        importances = rf.feature_importances_
        indices = np.argsort(importances)[::-1]
        return X.columns[indices[:k]].tolist()
    else:
        raise ValueError("Invalid method. Choose 'univariate', 'mutual_info', 'rfe', or 'random_forest'.")
    
    selector.fit(X, y)
    return X.columns[selector.get_support()].tolist()

In [17]:
def get_model():
    # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
    return LogisticRegression(max_iter=1000, random_state=RANDOM_SEED)

In [18]:

def perform_grid_search(X, y):
    param_grid = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga']
    }
    
    grid_search = GridSearchCV(
        LogisticRegression(max_iter=1000, random_state=RANDOM_SEED),
        param_grid,
        cv=5,
        scoring='f1',
        n_jobs=-1
    )
    
    grid_search.fit(X, y)
    
    # print("Best parameters:", grid_search.best_params_)
    # print("Best cross-validation score:", grid_search.best_score_)
    
    return grid_search.best_estimator_

In [19]:
def safe_balanced_accuracy(y_true, y_pred):
    try:
        return balanced_accuracy_score(y_true, y_pred)
    except ValueError:
        return float(y_true.iloc[0] == y_pred[0])

def evaluate_model_loocv(X, y, model):
    loo = LeaveOneOut()
    scores = []
    n_samples = X.shape[0]
    
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        score = safe_balanced_accuracy(y_test, y_pred)
        scores.append(score)
        
        if (len(scores) % 50 == 0) or (len(scores) == n_samples):
            print(f"Processed {len(scores)}/{n_samples} samples. Current average score: {np.mean(scores):.3f}")
    
    return np.mean(scores)

In [20]:
def evaluate_model_with_feature_selection(X, y, k, method='univariate', n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    
    precision_scores = []
    recall_scores = []
    f1_scores = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        selected_features = select_features(X_train, y_train, k, method)
        X_train_selected = X_train[selected_features]
        X_test_selected = X_test[selected_features]
        
        model = perform_grid_search(X_train_selected, y_train)
        
        y_pred = model.predict(X_test_selected)
        
        print("Precision:", precision_score(y_test, y_pred), "Recall:", recall_score(y_test, y_pred), "F1:", f1_score(y_test, y_pred))
        precision_scores.append(precision_score(y_test, y_pred))
        recall_scores.append(recall_score(y_test, y_pred))
        f1_scores.append(f1_score(y_test, y_pred))
    
    return precision_scores, recall_scores, f1_scores, selected_features

In [21]:
def compare_feature_selection_methods(X, y, methods, k_range):
    results = {}
    for method in methods:
        method_results = []
        for k in k_range:
            _, _, f1_scores, _ = evaluate_model_with_feature_selection(X, y, k, method)
            method_results.append(np.mean(f1_scores))
        results[method] = method_results
    return results

In [22]:
def train_and_evaluate_model(train_file, test_file=None, target_column='target', test_size=0.2):
    # Extract dataset name from the file path
    dataset_name = os.path.splitext(os.path.basename(train_file))[0]
    
    if test_file:
        X_train, y_train = load_and_preprocess_data(train_file, target_column, test_size=0)
        X_test, y_test = load_and_preprocess_data(test_file, target_column, test_size=0)
    else:
        X_train, X_test, y_train, y_test = load_and_preprocess_data(train_file, target_column, test_size=test_size)
    
    # Calculate the ideal test condition using Leave-One-Out
    best_model = perform_grid_search(X_train, y_train)
    ideal = evaluate_model_loocv(X_train, y_train, best_model)
    print('Ideal (Leave-One-Out): %.3f' % ideal)
    
    # Compare different feature selection methods
    methods = ['univariate', 'mutual_info', 'rfe', 'random_forest']
    k_range = range(1, len(X_train.columns) + 1)
    results = compare_feature_selection_methods(X_train, y_train, methods, k_range)
    
    # Create 'plots' directory if it doesn't exist
    os.makedirs('plots', exist_ok=True)
    
    # Plot comparison of feature selection methods
    plt.figure(figsize=(12, 6))
    for method in methods:
        plt.plot(k_range, results[method], label=method)
    plt.axhline(y=ideal, color='r', linestyle='--', label='Ideal (Leave-One-Out)')
    plt.xlabel('Number of Features')
    plt.ylabel('Mean F1 Score')
    plt.title(f'Comparison of Feature Selection Methods - {dataset_name}')
    plt.legend()
    plt.savefig(f'plots/feature_selection_comparison_{dataset_name}.png')
    plt.show()
    
    # Find best method and k
    best_method = max(results, key=lambda x: max(results[x]))
    best_k = k_range[np.argmax(results[best_method])]
    print(f"Best method: {best_method}")
    print(f"Best number of features: {best_k}")
    
    # Evaluate with best method and k
    precision_scores, recall_scores, f1_scores, best_features = evaluate_model_with_feature_selection(X_train, y_train, best_k, best_method)
    
    # Plot k-fold cross-validation results
    folds = range(1, 11)
    plt.figure(figsize=(12, 6))
    plt.errorbar(folds, f1_scores, fmt='o-', label='F1 Score')
    plt.axhline(y=ideal, color='r', linestyle='--', label='Ideal (Leave-One-Out)')
    plt.xlabel('Fold')
    plt.ylabel('F1 Score')
    plt.title(f'10-Fold Cross-Validation Results - {dataset_name} (k={best_k}, method={best_method})')
    plt.legend()
    plt.savefig(f'plots/cross_validation_results_{dataset_name}.png')
    plt.show()
    
    print("\nFinal Results:")
    print(f"Mean Precision: {np.mean(precision_scores):.3f}")
    print(f"Mean Recall: {np.mean(recall_scores):.3f}")
    print(f"Mean F1 score: {np.mean(f1_scores):.3f}")
    print(f"Best features: {best_features}")
    
    # Train final model and evaluate on test set
    X_train_selected = X_train[best_features]
    X_test_selected = X_test[best_features]
    
    final_model = perform_grid_search(X_train_selected, y_train)
    
    print("\nEvaluating Challenge Dataset")
    y_pred = final_model.predict(X_test_selected)
    
    print("\nChallenge Dataset Results:")
    print(f"Precision: {precision_score(y_test, y_pred):.3f}")
    print(f"Recall: {recall_score(y_test, y_pred):.3f}")
    print(f"F1 Score: {f1_score(y_test, y_pred):.3f}")
    
    return final_model, best_features

In [ ]:
if __name__ == "__main__":
    set_seed()
    train_file = 'data/cleveland.csv'
    test_file = 'data/cleveland-test-sample.csv'
    target_column = 'disease'
    
    trained_model, selected_features = train_and_evaluate_model(train_file, test_file, target_column)

#### Part 2: Diabetes Data
Sourced from CDC BRFSS 2015 https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset?resource=download

In [ ]:
if __name__ == "__main__":
    set_seed()
    train_file = 'data/diabetes_sample.csv'
    test_file = None
    target_column = 'Diabetes_binary'
    
    trained_model, selected_features = train_and_evaluate_model(train_file, test_file, target_column)